In [2]:
from pprint import pprint
from datetime import datetime
from tqdm import tqdm
import matplotlib
import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd
from numpy.lib.stride_tricks import sliding_window_view
from utils import *
from utils import markdown

def reduce_repeated_elements(arr):
    mask = np.concatenate(([True], np.diff(arr) != 0))
    reduced_arr = arr[mask]
    return reduced_arr

def compute_transition_probabilities(arr, normalized=True):
    num_integers = np.max(arr) + 1
    transition_matrix = np.zeros((num_integers, num_integers), dtype=[int,float][int(normalized)])
    # Create an array with shifted elements
    shifted_arr = np.roll(arr, -1)
    # Count occurrences of transitions
    unique_transitions, counts = np.unique(arr * num_integers + shifted_arr, return_counts=True)
    transition_matrix[unique_transitions // num_integers, unique_transitions % num_integers] = counts
    # Normalize transition matrix
    if normalized:
        transition_matrix /= np.sum(transition_matrix, axis=1, keepdims=True)
    return transition_matrix

def task_transition_matrix(eye_data, use_none=False, normalized=False):
    for i, (task, window_properties) in enumerate(sorted(ALL_WINDOW_PROPERTIES.items()), start=int(use_none)):
        intaskbox = in_box(eye_data['x'].to_numpy(), eye_data['y'].to_numpy(), window_properties['position'], window_properties['size'])
        eye_data.loc[intaskbox, 'task'] = i  
    task_data = reduce_repeated_elements(eye_data['task'].to_numpy())
    task_t_probs = compute_transition_probabilities(task_data, normalized=False)
    labels = (["none"] if use_none else []) + list(sorted(ALL_WINDOW_PROPERTIES.keys()))
    return pd.DataFrame(task_t_probs, index=labels, columns=labels)
    
#given a np array of shape (n,2) that contains intervals of time with an associated label (1-m) and a total duration of D seconds, how would I discretize this into an array with each element representing  elements using 0 to stand for values that were not in any interval, and the associated label to stand for any element that is
def data_iterator():
    data = load_nested_dict('data/Processed', ignore=IGNORE_NO_EYETRACKING)
    for i, (participant, _data1) in enumerate(data.items()):
        for j, (experiment, _data2) in enumerate(_data1.items()):
            yield (i, participant), (j, experiment[3:]), _data2
    
def get_eye_tracking_tasks(data):
    data['task'] = np.array(['none'] * data.shape[0])
    for key, window_properties in ALL_WINDOW_PROPERTIES.items():
        isin = in_box(data['x'], data['y'], window_properties['position'], window_properties['size'])
        data.loc[isin, 'task'] = key
    return data


def merge_intervals_nonoverlapping(intervals, labels=None):
    if labels is None:
        labels = np.arange(len(intervals))
    labels = np.concatenate([np.full((x.shape[0],1),i) for x, i in zip(intervals, labels)])
    intervals = np.concatenate(intervals, axis=0)
    indx = np.argsort(intervals[:,0]) # sort by start times
    intervals, labels = intervals[indx], labels[indx]
    
    return pd.DataFrame(dict(t1 = intervals[:,0], t2=intervals[:,1], task=labels[:,0]))
    
def in_intervals(intervals, points):
    return (points >= intervals[:, 0][:, np.newaxis]) & (points <= intervals[:, 1][:, np.newaxis])

def plot_intervals(intervals, ax, alpha=0.25, color='red'):
    for interval in intervals:
        ax.axvspan(*interval, alpha=alpha, color=color, linewidth=0)

def transition_pattern_count(data, n=2):
    from numpy.lib.stride_tricks import sliding_window_view
    tp = sliding_window_view(data, (n,)).squeeze()
    u, c = np.unique(tp, axis=0, return_counts=True)
    columns = [f"t{i}" for i in range(n)] + ["count"]
    return pd.DataFrame(data=np.concatenate([u,c[:,np.newaxis]], axis=1), columns=columns)

def in_during_still(intervals1, intervals2): # computes a dataframe tells us about the overlap of interval1 and interval2
     # if the start of the gaze is within a failure interval, it was already failed when they looked.
    fstart = in_intervals(intervals1, intervals2['t1'].to_numpy()).any(axis=0)
    # if the start of the failure is within the gaze interval, it started during the gaze.
    fduring = in_intervals(intervals2[['t1','t2']].to_numpy(), intervals1[:,0]).any(axis=1) 
    # if the end of the gaze is in a failure interval, it continues after they look away.
    fend = in_intervals(intervals1, intervals2['t2'].to_numpy()).any(axis=0) 
    # (fstart OR fduring) AND NOT fend will tell us that the failure was resolved during the look
    return pd.DataFrame(dict(fstart=fstart, fduring=fduring, fend=fend))

def compute_failure_intervals(data, task, start_time, finish_time):
    data = data['tasks'][task]
    return merge_intervals([compute_time_intervals(x.failure, x.timestamp, start_time, finish_time).intervals for x in data.values()])

def compute_warning_intervals(data, task, start_time, finish_time):
    data = data['warnings'][task]
    return compute_time_intervals(data['value'].astype(int), data['timestamp'], start_time, finish_time).intervals
    
def compute_eyetracking_intervals(data, task, start_time, finish_time, gaze_only=False):
    data = data['eye_tracking']
    window_properties = ALL_WINDOW_PROPERTIES[task]
    # compute gaze intervals
    intaskbox = in_box(data['x'].to_numpy(), data['y'].to_numpy(), window_properties['position'], window_properties['size'])
    if gaze_only: # USE THIS IF WE ONLY WANT GAZE
        gaze = data['gaze'].to_numpy().astype(bool)
        intaskbox = np.logical_and(intaskbox, gaze) 
    return compute_time_intervals(intaskbox, data['timestamp'].to_numpy(), start_time, finish_time, pad='next').intervals

def get_interval_data(_data2):
    start_time, finish_time =  _data2['start_time'], _data2['finish_time']
    tasks = sorted(_data2['tasks'].keys()) # ordering of tasks so that we can index later, keep them sorted.
    # compute failure intervals
    failure_intervals = [compute_failure_intervals(_data2, task, start_time, finish_time) for task in tasks]
    # compute eyetracking intervals
    eyetracking_intervals = [compute_eyetracking_intervals(_data2, task, start_time, finish_time, gaze_only=GAZE_ONLY) for task in tasks]
    # compute warning intervals
    warning_intervals = [compute_warning_intervals(_data2, task, start_time, finish_time) for task in tasks]
    # combine eyetracking intervals labelling each row with the task
    # | t1 | t2 | task |: tasks that the participant is looking at and when.
    df_eyetracking = merge_intervals_nonoverlapping(eyetracking_intervals)

    # compute failure data, check when failures are happening and where the participant is looking
    # this dataframe will contain bools indicating start,in,during for each row in df_eyetracking
    # | START | IN | DURING |
    df_failure = [in_during_still(fintervals, df_eyetracking) for fintervals in failure_intervals]
    # do the same for warnings.
    df_warning = [in_during_still(wintervals, df_eyetracking) for wintervals in warning_intervals]
    return SimpleNamespace(df_warning=df_warning, df_failure=df_failure, df_eyetracking=df_eyetracking, 
                           failure_intervals=failure_intervals, eyetracking_intervals=eyetracking_intervals,
                           warning_intervals=warning_intervals, tasks=tasks)

def get_counts(df_eyetracking, indx, tasks):
    return df_eyetracking.iloc[indx]['task'].value_counts().reindex(range(len(tasks)), fill_value=0).to_numpy()


    
    

In [5]:
stats = {}
data = load_nested_dict('data/Processed', ignore=IGNORE_NO_EYETRACKING) 
for participant, _data1 in data.items():
    stats[participant] = dict()
    for experiment, _data2 in _data1.items():
        result = get_interval_data(_data2)
        #print(result)
        
        
        
        
        

In [7]:

GAZE_ONLY = False 



pd.set_option("display.max_rows", 1000)

stats = dict()


data = load_nested_dict('data/Processed', ignore=IGNORE_NO_EYETRACKING) 
for participant, _data1 in data.items():
    stats[participant] = dict()
    for experiment, _data2 in _data1.items():
        exp_stats = stats[participant][experiment[3:]] = dict()
        result = get_interval_data(_data2)
        # COMPUTE SOME STATISTICS!
        
        # Q1: how often are participants looking at a task when it fails
        # NOTE: tasks can also fail when the participant is not looking at any task.
        def _gen():
            for task, dff in zip(result.tasks, result.df_failure):
                indx, tindx = np.arange(dff.shape[0])[dff['fduring']], result.tasks.index(task)
                counts = get_counts(result.df_eyetracking, indx, result.tasks)
                yield task, *counts
        looking_at_when_fails = pd.DataFrame(_gen(), columns=['looking_at', *result.tasks])
        exp_stats['looking_at_when_fails'] = looking_at_when_fails
        
        for task, dff in zip(result.tasks, result.df_failure):
            # whether the failure was resolve during a look.
            dff['fresolved']  = np.logical_or(dff['fstart'].to_numpy(), dff['fduring'].to_numpy())
            dff['fresolved']  = np.logical_and(dff['fresolved'], np.logical_not(dff['fend']))
            dff['looking_at'] = result.df_eyetracking['task']
            dff['task'] = result.tasks.index(task)
        
        #print(experiment)
       
        exp_stats['resolved_while_looking'] = pd.DataFrame(columns=['task', 'resolved', 'total', 'resolved/total'])
        for task, dff in zip(result.tasks, result.df_failure):
            # Q2: how often was the task resolved while the participant was looking at it?
            resolved = dff[dff['fresolved']]
            resolved_while_looking = resolved['task'] == resolved['looking_at']
            #print(f"{f'Resolved {task} while looking':<33} : {resolved_while_looking.sum()}/{resolved.shape[0]}")
            exp_stats['resolved_while_looking'].loc[len(exp_stats['resolved_while_looking'])] = [task, resolved_while_looking.sum(), resolved.shape[0], f"{resolved_while_looking.sum()}/{resolved.shape[0]}"]
        
        exp_stats['task_swaps'] = pd.DataFrame(columns=['task', 'swaps'])# 'mean', 'std', 'min', 'max'])
        # Q3: how many task swaps before resolving a failure
        for task, dff in zip(result.tasks, result.df_failure):
            failing = dff[np.logical_or(dff['fresolved'], dff['fend'])]
            indx = np.array([0] + (np.arange(failing.shape[0])[failing['fresolved']] + 1).tolist())
            swaps = np.array((indx[1:] - indx[:-1]) - 1) # counting transitions not intervals
            exp_stats['task_swaps'].loc[len(exp_stats['task_swaps'])] = [task, swaps]
            #print(f"Number of task swaps for each failure: {swaps}")
            
        #for task, dfw in zip(result.tasks, result.df_warning):
        #    if dfw[['fstart', 'fduring', 'fend']].any().any(): # there was some kind of warning
        #        dfw['looking_at'] = result.df_eyetracking['task']
        #        for t, dff in [(t, (dff['fstart'] | dff['fduring']).to_numpy()) for dff,t in zip(result.df_failure, result.tasks)]:
        #            dfw[f"f{t}"] = dff
                #print(task)
                #print(dfw)
            
        tr2 = transition_pattern_count(result.df_eyetracking['task'], 2)
        tr2[['t0', 't1']] = np.array(result.tasks)[tr2[['t0', 't1']].to_numpy()]
        tr3 = transition_pattern_count(result.df_eyetracking['task'], 3)
        tr3[['t0', 't1', 't2']] = np.array(result.tasks)[tr3[['t0', 't1', 't2']].to_numpy()]
        exp_stats['tr3'] = tr3
        exp_stats['tr2'] = tr2
        
        
        #print(participant)
        #markdown(f"##### 2-gram transitions\n {_data[experiment[3:]]['tr2'].to_markdown()}")
        #markdown(f"##### 3-gram transitions\n {_data[experiment[3:]]['tr3'].to_markdown()}")   

from pprint import pprint
pprint(stats['P01'])

{'icuA': {'looking_at_when_fails':   looking_at  fuel  system  tracking
0       fuel     2       0         0
1     system     4       0         3
2   tracking     0       0         0,
          'resolved_while_looking':        task  resolved  total resolved/total
0      fuel         2      2            2/2
1    system         7      8            7/8
2  tracking         0      0            0/0,
          'task_swaps':        task                      swaps
0      fuel                     [0, 0]
1    system  [1, 2, 6, 1, 1, 5, 15, 7]
2  tracking                         [],
          'tr2':          t0        t1  count
0      fuel      fuel      4
1      fuel    system     18
2      fuel  tracking     49
3    system      fuel     15
4    system    system      2
5    system  tracking     54
6  tracking      fuel     53
7  tracking    system     50
8  tracking  tracking      9,
          'tr3':           t0        t1        t2  count
0       fuel      fuel    system      1
1       fuel     

In [ ]:
fp = Statistics.compute_failure_proportion_statistics()
fl = Statistics.compute_failure_length_statistics()
fc = Statistics.compute_failure_count_statistics()


In [19]:


df = fc
df['difficulty'] = df['experiment'].str[-1]
dfB = df[df['difficulty'] == 'B']

print(dfB)

   participant experiment  fuel  system  tracking  total difficulty
1          P01       icuB    15      44         0     59          B
3          P01      icuaB     8      45         0     53          B
5          P02       icuB     5      46         8     59          B
7          P02      icuaB     9      44        18     71          B
9          P03       icuB     0      15         0     15          B
11         P03      icuaB     4      45        11     60          B
13         P04       icuB     6      46        16     68          B
15         P04      icuaB     7      41         7     55          B
17         P05       icuB    10      35        10     55          B
19         P05      icuaB    13      36        21     70          B
21         P06       icuB    13      44         5     62          B
23         P06      icuaB     5      43         4     52          B
25         P07       icuB     7      45        23     75          B
27         P07      icuaB    11      38        2

In [12]:
scores = fp

for participant in scores['participant'].unique():
    _data = stats[participant]
    experiments = list(sorted(_data.keys(), key=lambda k:str(reversed(k))))
    markdown(f"## {participant}")
    markdown(f"#### Q1: How often did a task fail while the participant was looking at it?")
    tables = [_data[exp]['looking_at_when_fails'].to_markdown() for exp in experiments]
    embed_markdown_tables(tables, ncols=len(experiments), headers=experiments)
    markdown("NOTE: tasks can also fail when the participant is not looking at any task, this is not recorded here. TODO should it be?")
      
    markdown(f"#### Q2: How often was a task resolved while the participant was looking at it?")
    tables = [_data[exp]['resolved_while_looking'][['task', 'resolved/total']].to_markdown() for exp in experiments]
    embed_markdown_tables(tables, ncols=len(experiments), headers=experiments)

    markdown(f"#### Q3: how many task swaps before resolving a failure?")
    #tables = [_data[exp]['task_swaps'].to_markdown() for exp in experiments]
    #embed_markdown_tables(tables, ncols=len(experiments), headers=experiments)
    #swaps.mean(), swaps.std(), swaps.min() if swaps.shape[0] > 0 else float('nan'), swaps.max() if swaps.shape[0] > 0 else float('nan')
    
    
 
    

## P01

#### Q1: How often did a task fail while the participant was looking at it?

NOTE: tasks can also fail when the participant is not looking at any task, this is not recorded here. TODO should it be?

#### Q2: How often was a task resolved while the participant was looking at it?

#### Q3: how many task swaps before resolving a failure?

...

## P25

#### Q1: How often did a task fail while the participant was looking at it?

NOTE: tasks can also fail when the participant is not looking at any task, this is not recorded here. TODO should it be?

#### Q2: How often was a task resolved while the participant was looking at it?

#### Q3: how many task swaps before resolving a failure?

In [4]:
        
        # Q: Transition probabilities
        #t_probs = compute_transition_probabilities(df_eyetracking['task'].to_numpy(), normalized=False)
        #markdown("### Task Transition Tables")
        #markdown(pd.DataFrame(t_probs, index=tasks, columns=tasks).to_markdown())
        tr2 = transition_pattern_count(result.df_eyetracking['task'], 2)
        tr2[['t0', 't1']] = np.array(result.tasks)[tr2[['t0', 't1']].to_numpy()]
        #markdown(f"##### 2-gram transitions\n {tr2.to_markdown()}")
        tr3 = transition_pattern_count(result.df_eyetracking['task'], 3)
        tr3[['t0', 't1', 't2']] = np.array(result.tasks)[tr3[['t0', 't1', 't2']].to_numpy()]
        #markdown(f"##### 3-gram transitions\n {tr3.to_markdown()}")    
        # NOTE: participants transition to and from tracking a lot more.
        # They also seem to do loops of the three tasks, since tracking is kind of on the way, it will be seen more
        # TODO do all participants follow this pattern? probably... its quite efficient!
        
    break
    
# TODO compare mean interval length with and without agents... maybe it improves???



# NOTE: perhaps the agents can improve performance by guiding them to tasks that are at risk of failure...? 
# this might be a bit aggressive... 

IndentationError: unexpected indent (2165638889.py, line 32)

In [ ]:
# COMPUTE TRANSITION PROBABILITIES FOR EYES 
# NOTE: this will give a different result to the above because it only uses gaze data and removes self transitions.
data = load_nested_dict('data/Processed') 
for participant, _data1 in data.items(): #itertools.islice(data.items(), 1):
    for experiment, _data2 in _data1.items():
        start_time, finish_time =  _data2['start_time'], _data2['finish_time']
        eye_data = _data2['eye_tracking']  
        gaze = eye_data['gaze'].to_numpy().astype(bool)
        eye_data['task'] = np.zeros_like(eye_data['x'].to_numpy(), dtype=int)
        eye_data = eye_data.loc[gaze] # take only gaze data.                 
        t_probs = task_transition_matrix(eye_data, use_none=False, normalized=False)
        print(t_probs)
        #gaze = eye_data['gaze'].to_numpy()
    break

        